In [ ]:
import platform, subprocess, sys, torch
print('Python', sys.version)
print('PyTorch', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))
    print('Compute capability OK')


Python 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch 2.8.0+cu126
CUDA available: True
GPU: NVIDIA A100-SXM4-40GB
Compute capability OK


In [ ]:
!pip -q install --upgrade pip
!pip -q install torch --index-url https://download.pytorch.org/whl/cu121 -U
!pip -q install transformers accelerate datasets peft trl bitsandbytes sentencepiece safetensors evaluate
!pip -q install huggingface_hub
!pip -q install xformers --only-binary=:all: -U
import torch, transformers, peft, datasets, accelerate, bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.2 MB/s eta 0:00:00


In [ ]:
from dataclasses import dataclass
from google.colab import drive
drive.mount('/content/drive')

@dataclass
class Config:
    base_model: str = 'mistralai/Mistral-7B-Instruct-v0.2'
    output_dir: str = '/content/drive/My Drive/aalap-qlora-out-mask'
    adapter_name: str = 'aalap-qlora'
    dataset_path_or_files: str | list[str] = 'opennyaiorg/aalap_instruction_dataset'  # set to your JSONL/Parquet path(s) in Drive or Hub
    use_flash_attn: bool = False  # Colab safety; enable if your environment supports it
    push_to_hub: bool = False
    hub_repo: str | None = None  # e.g. 'your-username/Aalap-Mistral-7B-qlora'
    hub_private: bool = True
    # QLoRA & training params
    r: int = 16
    lora_alpha: int = 32
    lora_dropout: float = 0.05
    target_modules: tuple = ('q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj')
    load_in_4bit: bool = True
    nf4: bool = True
    lr: float = 2e-5
    weight_decay: float = 0.01
    warmup_ratio: float = 0.03
    num_train_epochs: float = 2  # Enough for QLoRA fine-tune on Colab
    per_device_train_batch_size: int = 32  # A100 40GB can handle this
    gradient_accumulation_steps: int = 1
    logging_steps: int = 10
    save_steps: int = 200
    max_seq_length: int = 4096  # Aalap supports up to 32k; Colab memory limits apply
    eval_steps: int = 0

cfg = Config()
cfg

Mounted at /content/drive


Config(base_model='mistralai/Mistral-7B-Instruct-v0.2', output_dir='/content/drive/My Drive/aalap-qlora-out-mask', adapter_name='aalap-qlora', dataset_path_or_files='opennyaiorg/aalap_instruction_dataset', use_flash_attn=False, push_to_hub=False, hub_repo=None, hub_private=True, r=16, lora_alpha=32, lora_dropout=0.05, target_modules=('q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'down_proj', 'up_proj'), load_in_4bit=True, nf4=True, lr=2e-05, weight_decay=0.01, warmup_ratio=0.03, num_train_epochs=2, per_device_train_batch_size=32, gradient_accumulation_steps=1, logging_steps=10, save_steps=200, max_seq_length=4096, eval_steps=0)

In [ ]:
## 3) Data loading / preprocessing
from typing import Iterable
from datasets import load_dataset, Dataset, DatasetDict
import os, json
from huggingface_hub import login
login("") # Token hidden for security


def load_legal_instruct_dataset(path_or_files) -> DatasetDict | Dataset:
    """Load one or more JSONL files or a Hugging Face dataset ID."""
    if not path_or_files:
        # Fallback tiny demo dataset
        rows = [
            {"instruction": "Identify the legal issue.", "input": "A landlord refused to return a security deposit.", "response": "Potential issue: wrongful withholding of security deposit under tenancy law."},
            {"instruction": "Summarize this legal passage.", "input": "The contract stipulated arbitration in case of disputes.", "response": "The parties agreed to resolve disputes via arbitration."},
        ]
        return Dataset.from_list(rows)

    # ✅ NEW: Handle Hugging Face dataset names
    if isinstance(path_or_files, str) and not os.path.exists(path_or_files):
        try:
            ds = load_dataset(path_or_files)
            # 💡 FIX: Return the entire DatasetDict, not just the train split
            return ds
        except Exception as e:
            raise ValueError(f"Could not load dataset '{path_or_files}': {e}")

    # ✅ OLD behavior: local JSONL files
    if isinstance(path_or_files, str):
        path_or_files = [path_or_files]
    data = []
    for p in path_or_files:
        with open(p, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip():
                    data.append(json.loads(line))
    return Dataset.from_list(data)


# --- Load the full DatasetDict ---
raw_ds_dict = load_legal_instruct_dataset(cfg.dataset_path_or_files)

# Check if it's a dict (from HF) or a single Dataset (from local files)
if isinstance(raw_ds_dict, DatasetDict) and "test" in raw_ds_dict:
    print("Loaded DatasetDict with train/test splits.")
    # --- Process Train Split ---
    train_ds = raw_ds_dict["train"].shuffle(seed=42)

    # --- Process Test/Eval Split ---
    eval_ds = raw_ds_dict["test"].shuffle(seed=42)

    print(f"Using {len(train_ds)} training samples and {len(eval_ds)} evaluation samples.")

else:
    # Fallback for local files: split the single dataset
    print("Loaded single dataset, splitting 90/10 for train/eval.")
    full_ds = raw_ds_dict.shuffle(seed=42)

    split_ds = full_ds.train_test_split(test_size=0.1, seed=42)
    train_ds = split_ds["train"]
    eval_ds = split_ds["test"]
    print(f"Using {len(train_ds)} training samples and {len(eval_ds)} evaluation samples.")


### 3.a) Supervised fine-tuning formatting

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(cfg.base_model, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def format_row_as_chat(example):
    user_content = example.get("instruction", "")
    if example.get("input"):
        user_content += "\n\n" + str(example["input"])
    messages = [
        {"role": "system", "content": "You are Aalap, a helpful Indian legal assistant focusing on legal reasoning."},
        {"role": "user", "content": user_content},
    ]
    example["messages"] = messages
    # 💡 --- CRITICAL FIX --- 💡
    # The answer column is 'response', not 'output'
    example["labels"] = example.get("response", "")
    return example

# --- Map both train and eval datasets ---
chat_train_ds = train_ds.map(format_row_as_chat)
chat_eval_ds = eval_ds.map(format_row_as_chat)

print("\nFormatted chat datasets ready:")
print(chat_train_ds)
print(chat_eval_ds)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

data/train-00000-of-00001-5242b1cfcb40f3(…):   0%|          | 0.00/92.9M [00:00<?, ?B/s]

data/test-00000-of-00001-c4653fdde527ae7(…):   0%|          | 0.00/4.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21178 [00:00<?, ? examples/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Map:   0%|          | 0/21178 [00:00<?, ? examples/s]

Map:   0%|          | 0/1094 [00:00<?, ? examples/s]


Formatted chat datasets ready:
Dataset({
    features: ['input_text', 'system_prompt', 'user_prompt', 'output_text', 'task', 'combined_input_prompt', 'messages', 'labels'],
    num_rows: 21178
})
Dataset({
    features: ['input_text', 'system_prompt', 'user_prompt', 'output_text', 'task', 'combined_input_prompt', 'messages', 'labels'],
    num_rows: 1094
})


In [ ]:
from transformers import DataCollatorForLanguageModeling

# Standard data collator for causal LM (no masked LM)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
## Adjust data preparation or training loop
# --- Full training cell (version-stable, with ANSWER-ONLY LOSS MASKING) ---

from peft import LoraConfig, TaskType, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    # default_data_collator,   # <-- use this (keeps our labels) # REMOVED
    DataCollatorForLanguageModeling # ADDED
)
import torch, inspect
from transformers.trainer_callback import EarlyStoppingCallback

# ---- LoRA + 4-bit config ----
bnb_config = BitsAndBytesConfig(
    load_in_4bit=cfg.load_in_4bit,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    bnb_4bit_quant_type='nf4' if cfg.nf4 else 'fp4',
)

lora_config = LoraConfig(
    r=cfg.r,
    lora_alpha=cfg.lora_alpha,
    lora_dropout=cfg.lora_dropout,
    target_modules=list(cfg.target_modules),
    task_type=TaskType.CAUSAL_LM,
)

# ---- Tokenizer (load if not already present) ----
try:
    tokenizer  # noqa: F821
except NameError:
    tokenizer = AutoTokenizer.from_pretrained(cfg.base_model, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ---- Model ----
# Use dtype= (not torch_dtype=) to avoid deprecation
model = AutoModelForCausalLM.from_pretrained(
    cfg.base_model,
    quantization_config=bnb_config if cfg.load_in_4bit else None,
    dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map='auto',   # <-- fixed (removed stray 'z')
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# ---- Dataset formatting (with ANSWER-ONLY label masking) ----
def chat_template(messages):
    # Minimal template: [SYSTEM] + [USER] → model continues at [ASSISTANT]
    text = ""
    for m in messages:
        if m["role"] == "system":
            text += f"<s>[SYSTEM]\n{m['content']}\n"
        elif m["role"] == "user":
            text += f"[USER]\n{m['content']}\n"
        elif m["role"] == "assistant":
            text += f"[ASSISTANT]\n{m['content']}\n"
    text += "[ASSISTANT]\n"
    return text

def tokenize_answer_only(example):
    # Build prompt and answer
    prompt = chat_template(example["messages"])
    answer = example.get("labels", "")  # your target text

    # Tokenize separately to know split point; no extra specials
    prompt_enc = tokenizer(prompt, add_special_tokens=False)
    answer_enc = tokenizer(answer, add_special_tokens=False)

    input_ids = prompt_enc["input_ids"] + answer_enc["input_ids"]
    attention_mask = prompt_enc["attention_mask"] + answer_enc["attention_mask"]

    # Labels: ignore prompt tokens (-100), learn on answer tokens
    labels = [-100] * len(prompt_enc["input_ids"]) + answer_enc["input_ids"]

    # Truncate consistently
    if len(input_ids) > cfg.max_seq_length:
        input_ids = input_ids[:cfg.max_seq_length]
        attention_mask = attention_mask[:cfg.max_seq_length]
        labels = labels[:cfg.max_seq_length]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

# --- Tokenize both train and eval datasets ---
# (chat_train_ds, chat_eval_ds must be defined in your previous cell)
tokenized_train = chat_train_ds.map(
    tokenize_answer_only,
    remove_columns=[c for c in chat_train_ds.column_names if c not in ("input_ids", "attention_mask", "labels")]
)

tokenized_eval = chat_eval_ds.map(
    tokenize_answer_only,
    remove_columns=[c for c in chat_eval_ds.column_names if c not in ("input_ids", "attention_mask", "labels")]
)

# ---- TrainingArguments (compatible across versions) ----
ta_kwargs = dict(
    output_dir=cfg.output_dir,
    per_device_train_batch_size=cfg.per_device_train_batch_size,
    gradient_accumulation_steps=cfg.gradient_accumulation_steps,
    learning_rate=cfg.lr,
    num_train_epochs=cfg.num_train_epochs,
    logging_steps=cfg.logging_steps,
    bf16=torch.cuda.is_available(),
    lr_scheduler_type='cosine',
    warmup_ratio=cfg.warmup_ratio,
    weight_decay=cfg.weight_decay,
    gradient_checkpointing=False,   # set True if you want checkpointing
    report_to="none",
)

# --- Compatibility shim for eval strategy naming ---
sig = inspect.signature(TrainingArguments.__init__)
if "evaluation_strategy" in sig.parameters:
    ta_kwargs["evaluation_strategy"] = "steps"
elif "eval_strategy" in sig.parameters:
    ta_kwargs["eval_strategy"] = "steps"

# Shared across versions
ta_kwargs["eval_steps"] = 50
ta_kwargs["save_strategy"] = "steps"
ta_kwargs["save_steps"] = ta_kwargs["eval_steps"]
ta_kwargs["load_best_model_at_end"] = True
ta_kwargs["metric_for_best_model"] = "eval_loss"
ta_kwargs["greater_is_better"] = False

args = TrainingArguments(**ta_kwargs)

# ---- Data Collator (use DataCollatorForLanguageModeling) ---- # MODIFIED
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# ---- Trainer (use our new data collator) ---- # MODIFIED
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,  # <-- important # MODIFIED
    callbacks=[
       EarlyStoppingCallback(early_stopping_patience=3)
    ],
)

# ---- Train & save adapter ----
trainer.train()
trainer.save_model(cfg.output_dir)
tokenizer.save_pretrained(cfg.output_dir)
print("Saved adapter to", cfg.output_dir)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

trainable params: 41,943,040 || all params: 7,283,675,136 || trainable%: 0.5758


Map:   0%|          | 0/21178 [00:00<?, ? examples/s]

Map:   0%|          | 0/1094 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
50,0.551200,0.527454
100,0.026100,0.000353
150,0.000000,0.000033
200,0.000000,0.000014
250,0.000000,0.000010
300,0.000000,0.000008
350,0.000000,0.000007
400,0.000000,0.000006
450,0.000000,0.000005
500,0.000000,0.000004


Saved adapter to /content/drive/My Drive/aalap-qlora-out-mask


In [ ]:
!pip -q install transformers peft accelerate bitsandbytes

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TextStreamer
)
from peft import PeftModel
from google.colab import drive
import os

# --- 1. Mount Google Drive (if not already mounted) ---
drive_path = "/content/drive"
if not os.path.exists(drive_path):
    print("Mounting Google Drive...")
    drive.mount(drive_path)
else:
    print("Google Drive is already mounted.")

# --- 2. Define Model Paths (from your new config) ---
base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
adapter_path = "/content/drive/My Drive/aalap-qlora-out-mask"

# --- 3. Configure 4-bit Loading (BNB) ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type='nf4',
)

# --- 4. Load the Base Model ---
print(f"Loading base model: {base_model_id}")
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map='auto', # Automatically uses the A100
)

# --- 5. Load the Tokenizer ---
print(f"Loading tokenizer from: {adapter_path}")
tokenizer = AutoTokenizer.from_pretrained(adapter_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    # Set padding side to right for generation
    tokenizer.padding_side = "right"

# --- 6. Load the LoRA Adapter ---
print(f"Loading adapter: {adapter_path}")
# This merges your adapter weights onto the base model
model = PeftModel.from_pretrained(model, adapter_path)
model.eval() # Set model to evaluation mode

print("✅ Model and tokenizer loaded successfully.")

Google Drive is already mounted.
Loading base model: mistralai/Mistral-7B-Instruct-v0.2


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading tokenizer from: /content/drive/My Drive/aalap-qlora-out
Loading adapter: /content/drive/My Drive/aalap-qlora-out
✅ Model and tokenizer loaded successfully.


In [ ]:
from transformers import TextStreamer

input_text = '''
""" S.N. Shankar, J.

(1) This appeal was referred to the Full Bench principally to determine the exact import and effect of the provisions of Section 13 of the Prevention of Food Adulteration Act, 1954 (Act 37 of 1954) as amended up to date (hereinafter called "the Act") in the context of the right of the accused to have the sample of the seized article of food examined by the Director of Central Food Laboratory, Calcutta, under the provisions of sub-section (2) of Section 13 and the effect of the time-lapse occurring during the process.

(2) In order to appreciate the point involved it will be appropriate to set out briefly the relevant facts of the case which have been noticed in detail in the referring order.

(3) The two respondents to this appeal.
Bishan Sarup and Raghbir Parshad were acquitted by Magistrate 1st Class, Delhi of the charges framed against them under section 7 read with section 16 of the Act.
Aggrieved from this order, the Municipal Corporation of Delhi came up in appeal to this Court.
These respondents were prosecuted on a complaint filed against them through the Municipal Prosecutor on October 3, 1962 with the allegations that on March 27, 1962 at 7.45 A.M, Food Inspector, Balraj Kocher, had gone to shop -No. 1386 situated in Kalan Mahal, Delhi where a board stating that cow's milk was sold at the shop was hanging.
After disclosing his identity the Food Inspector purchased 660 mls of cow's milk from Bishan Sarup for 56 paise, against receipt Exhibit Pa which was thumb-marked and signed by Bishan Sarup.
The milk was divided by the Food Inspector into three equal parts and each part was put in separate clean and dried phials.
16 drops of formalin were added in each phial and they were then sealed.
One of the phials was handed over to the accused, the other was retained by the Food Inspector and the third was delivered in the office of the Public Analyst by the Food Inspector.
By his report exhibit Pe dated March 31, 1962, the Public Analyst found that the contents of fat in the sample were 6.7% while the non-fatty solids were only 8.03 per cent as against the prescribed standard of 8.5%.
Due to deficiency of non-fatty solids the Public Analyast declared the milk to be adulterated.
This led to the filing of the complaint against both the respondents under Section 7 read with section 16 of the Act.

(4) At the trial both the respondents denied the allegations made against them.
Bishan Sarup contended that he was not employed at the shop and that on the day when the sample was taken he had gone to the shop of a 'halwai' at Kalan Masjid and purchased two seers of milk from him and while he was returning from there he was held up by four or five prersons in 'khakhi' uniforns who took him to the shop wherefrom he had purchased the milk and took a sample of the milk and secured his signatures on a piece of paper.
The other respondent Raghbir Parshad admitted that he was the owner of the shop referred to in the complaint but maintained that Bishan Sarup was not his employee.
He also denied that the sample of milk was taken from his shop.
During the pendency of the trial, after the lapse of about a year and ten months, on August 22, 1964, Bishan Sarup applied to the learned trial magistrate that the sample of the milk in possession of the Municipal Corporation be sent to the Director of Central Food Laboratory, Calcutta, for his analysis and report.
This application was at first rejected by the learned Magistrate but the order was subsequently reviewed on...
"""
sections of law:
"""['proviso to Section 13(5) of Prevention of Food Adulteration Act, 1954 (Act 37 of 1954)', 'S. 13 of Prevention of Food Adulteration Act, 1954 (Act 37 of 1954)', 'section 342 of Code of Criminal Procedure ', 'Section 11 of Prevention of Food Adulteration Act, 1954 (Act 37 of 1954)', 'section 13 of Prevention of Food Adulteration Act, 1954 (Act 37 of 1954)', 'sub-section (2) of Section 13 of Prevention of Food Adulteration Act, 1954 (Act 37 of 1954)', 'Section 7 read with section 16 of Prevention of Food Adulteration Act, 1954 (Act 37 of 1954)', 'section 248 of Code of Criminal Procedure ', 'S. 13(2) of Prevention of Food Adulteration Act, 1954 (Act 37 of 1954)', 'section 7 of Prevention of Food Adulteration Act, 1954 (Act 37 of 1954)', 'sub-section (3) of Section 13 of Prevention of Food Adulteration Act, 1954 (Act 37 of 1954)', 'Sub-section (2) of Section 13 of Prevention of Food Adulteration Act, 1954 (Act 37 of 1954)', 'section 16 of Prevention of Food Adulteration Act, 1954 (Act 37 of 1954)', 'sub-section (1) of Section 13 of Prevention of Food Adulteration Act, 1954 (Act 37 of 1954)', 'Section 13 of Prevention of Food Adulteration Act, 1954 (Act 37 of 1954)', 'sub-section (5) of Section 13 of Prevention of Food Adulteration Act, 1954 (Act 37 of 1954)', 'Section 10 of Prevention of Food Adulteration Act, 1954 (Act 37 of 1954)', 'section 9 of Prevention of Food Adulteration Act, 1954 (Act 37 of 1954)', 'proviso to sub-section (5) of Section 13 of Prevention of Food Adulteration Act, 1954 (Act 37 of 1954)']
"""
Issues:
"""['Import and effect of the provisions of Section 13 of the Prevention of Food Adulteration Act, 1954 in the context of the right of the accused to have the sample of the seized article of food examined by the Director of Central Food Laboratory', 'Effect of the time-lapse occurring during the process of examination of the seized article of food by the Director of Central Food Laboratory', 'Validity of the report of the Director of Central Food Laboratory and its admissibility as evidence in court', 'Allegations of sample tampering by the Food Inspector before sending it to the Director for analysis', 'Failure of the accused to send the sample given to him for analysis by the Director']
"""
'''

# Split the input into sections
case_facts = input_text.split("sections of law:")[0].strip()
sections_of_law = input_text.split("sections of law:")[1].split("Issues:")[0].strip()
issues = input_text.split("Issues:")[1].strip()

# Construct the prompt string directly, matching the training template
# Add a dummy response token after [ASSISTANT]\n to encourage the model to start generating
# prompt = (
#     "<s>[SYSTEM]\nYour role is that of an Indian lawyer practicing in Delhi High Court. You've been tasked with representing the Petitioner - Municipal Corporation Of Delhi in a case against the Respondent - Bishan Sarup Etc.. Your goal is to build a strong argument highlighting the legal shortcomings of the Respondent's actions. Make use of the case facts, legal issues, pertinent law sections, and any additional material provided.\n"
#     "[USER]\n"
#     "Case Facts:\n"
#     f"{case_facts}\n\n"
#     "Sections of Law:\n"
#     f"{sections_of_law}\n\n"
#     "Issues:\n"
#     f"{issues}\n"
#     "[ASSISTANT]\n" # Explicitly add the assistant prompt token
# )

prompt = """
	<s> [INST] <<SYS>> You are justicebot, an Indian law assistant, you answer the given question as you are an Indian law practicioner. Mention the laws which help in aiding the reasoning of the user prompt. Do not just grab the law based on the words used but rather by what the law is actually about<</SYS>>
My land is surrounded by my brother's land i.e. to enter my land he must give way and now he is not allowing me to enter my own land i.e. i have no right of way what can?[/INST]
"""


print("Constructed Prompt:")
print(prompt)

inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

# Use a streamer to see output as it's generated
streamer = TextStreamer(tokenizer, skip_prompt=True)

with torch.no_grad():
    # Experiment with generation parameters - slightly lower temperature, higher top_p, add repetition_penalty
    out = model.generate(**inputs,
                     max_new_tokens=1500,
                     do_sample=True,
                     temperature=0.5, # Slightly lower temperature for potentially more focused output
                     top_p=0.95,
                     repetition_penalty=1.2, # Add repetition penalty to discourage repeating prompt tokens
                     streamer=streamer,
                     pad_token_id=tokenizer.eos_token_id)

# The streamer prints the output directly, so manual decoding is not needed here.

Constructed Prompt:

	<s> [INST] <<SYS>> You are justicebot, an Indian law assistant, you answer the given question as you are an Indian law practicioner. Mention the laws which help in aiding the reasoning of the user prompt. Do not just grab the law based on the words used but rather by what the law is actually about<</SYS>>
My land is surrounded by my brother's land i.e. to enter my land he must give way and now he is not allowing me to enter my own land i.e. i have no right of way what can?[/INST]

It seems that the issue at hand pertains to a dispute over access to your property through another parcel of land owned by your brother. In such situations, there may be legal remedies available under Indian law based on the specific facts and circumstances. Two relevant provisions come to mind:

1) Easements by prescription or long user: According to Section 62 of The Indian Limitation Act, 1963, if any person has for twenty years peaceably possessed the real property of another person 